# `flood` Load Test Report


### Contents
1. [Test Summary](#Test-Summary)
2. [Tests](#Tests)
    1. [Test: eth_getStorageAt](#Test:-eth_getStorageAt)

### Report Generation

*This report was generated by `flood`. See the `flood report --help` command for report generation options. This report can be executed as a Python notebook using the `.ipynb` version of this file.*

In [ ]:
import IPython
import polars as pl
import toolstr
import tooltime

import flood

flood.user_io.styles = {}

In [ ]:
# parameters

test_paths = {
    'eth_getStorageAt': '/var/folders/ml/dymw9nkx51j7scsxc3qpqvl80000gp/T/tmpcljzupq2',
}

metrics = ['success', 'throughput', 'p50', 'p90', 'p99']

In [ ]:
# load data

test_payloads = {
    test_name: flood.load_single_run_test_payload(test_path)
    for test_name, test_path in test_paths.items()
}

results_payloads = {
    test_name: flood.load_single_run_results_payload(output_dir=test_path)
    for test_name, test_path in test_paths.items()
}

# Test Summary

In [ ]:
# test list

toolstr.print_text_box('Tests')
for t, test_name in enumerate(results_payloads.keys()):
    print(str(t + 1) + '.', test_name)

In [ ]:
# test durations

time_per_test = {}
time_per_condition = {}

for test_name in results_payloads:
    results = results_payloads[test_name]["results"]
    time_per_test[test_name] = 0
    for condition_name in results.keys():
        time_per_condition.setdefault(condition_name, 0)
        time = sum(results[condition_name]["actual_duration"]) + sum(
            results[condition_name]["final_wait_time"]
        )
        time_per_test[test_name] += time
        time_per_condition[condition_name] += time

toolstr.print_text_box('Total time')
toolstr.print(tooltime.timelength_to_phrase(int(sum(time_per_test.values()))))
print()

toolstr.print_text_box('Total time per test')
rows = list(time_per_test.items())
toolstr.print_table(rows, labels=['test', 'time (s)'])

toolstr.print_text_box('Total time per condition')
rows = list(time_per_condition.items())
toolstr.print_table(rows, labels=['condition', 'time (s)'])

# Tests

# Test: eth_getStorageAt

In [ ]:
# load test results

test_name = 'eth_getStorageAt'
results_payload = results_payloads[test_name]
results = results_payload['results']

In [ ]:
# show test metadata

toolstr.print_text_box(test_name + ' parameters')
flood.user_io.print_load_test_summary(results_payload['test'])
toolstr.print('- nodes tested:')
nodes_df = pl.from_records(list(results_payload['nodes'].values()))
toolstr.print_dataframe_as_table(nodes_df)

In [ ]:
# show result tables

flood.user_io.print_metric_tables(results, metrics=metrics, comparison=True)

In [ ]:
# show result figures

colors = flood.user_io.get_nodes_plot_colors(nodes=results_payload['nodes'])
flood.user_io.plot_load_test_results(
    test_name=test_name,
    outputs=results,
    latency_yscale_log=True,
    colors=colors,
)

In [ ]:
# show errors

toolstr.print_text_box('Error messages present in each test')
unique_errors = {}
for n, name in enumerate(results.keys()):
    unique_errors.setdefault(name, set())
    unique_errors[name] |= {
        error for error_list in results[name]['errors'] for error in error_list
    }
    print(name)
    for error in unique_errors[name]:
        print('-', error)
    if n != len(results) - 1:
        print()

In [ ]:
# show complete results

for name in results.keys():
    toolstr.print_text_box(name + " Complete Results")
    df = pl.DataFrame(results[name])
    df = df.drop(
        "status_codes",
        "errors",
        "first_request_timestamp",
        "last_request_timestamp",
        "last_response_timestamp",
    )
    IPython.display.display(df)